In [ ]:
pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 54.6 MB/s eta 0:00:00


In [ ]:
pip install timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00


In [ ]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.6 MB/s eta 0:00:00


In [ ]:
pip install pytorch-metric-learning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
import glob
import sys
import os
import time
import random
import math

# DATALOADER
import cv2
from PIL import Image
import numpy as np
import albumentations as A
import torchvision.transforms as T
from PIL import Image
import pandas as pd

# BUILDING MODEL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models  # Import ResNet-50

# TRAINING
from torch.utils.data import DataLoader, Dataset
import faiss
from tqdm import tqdm

# OTHER STUFF
import timm
from transformers import (get_linear_schedule_with_warmup,
                          get_constant_schedule,
                          get_cosine_schedule_with_warmup,
                          get_cosine_with_hard_restarts_schedule_with_warmup,
                          get_constant_schedule_with_warmup)
import gc
import transformers
from transformers import CLIPProcessor, CLIPVisionModel,  CLIPVisionConfig
from pytorch_metric_learning import losses
#import open_clip

# UTILS
#import utilities

# Load the ResNet-50 model
resnet50 = models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 146MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import os
import faiss
import copy
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from PIL import Image
from torchvision import transforms
import cv2
import time

class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0,
                 m=0.50, easy_margin=False, ls_eps=0.0, device=torch.device('cuda')):
        super(ArcMarginProduct, self).__init__()
        self.device = device
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=self.device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) ------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

class DenseCrossEntropy(nn.Module):
    def forward(self, x, target):
        x = x.float()
        target = target.float()
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)

        loss = -logprobs * target
        loss = loss.sum(-1)
        return loss.mean()

class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super(FocalLoss, self).__init__()
        self.gamma = gamma

    def forward(self, x, target):
        x = x.float()
        target = target.float()
        probs = torch.nn.functional.softmax(x, dim=-1)
        logprobs = torch.log(probs)

        loss = -logprobs * target * (1 - probs) ** self.gamma
        loss = loss.sum(-1)
        return loss.mean()

class ArcMarginProduct_subcenter(nn.Module):
    def __init__(self, in_features, out_features, k=3):
        super().__init__()
        self.weight = nn.Parameter(torch.FloatTensor(out_features*k, in_features))
        self.reset_parameters()
        self.k = k
        self.out_features = out_features

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

    def forward(self, features):
        cosine_all = F.linear(F.normalize(features), F.normalize(self.weight))
        cosine_all = cosine_all.view(-1, self.out_features, self.k)
        cosine, _ = torch.max(cosine_all, dim=2)
        return cosine

class ArcFaceLossAdaptiveMargin(nn.modules.Module):
    def __init__(self, margins, s=30.0, crit='ce'):
        super().__init__()
        if crit == 'ce':
            self.crit = DenseCrossEntropy()
        else:
            self.crit = FocalLoss()
        self.s = s
        self.margins = margins

    def forward(self, logits, labels, out_dim):
        ms = []
        ms = self.margins[labels.cpu().numpy()]
        cos_m = torch.from_numpy(np.cos(ms)).float().cuda()
        sin_m = torch.from_numpy(np.sin(ms)).float().cuda()
        th = torch.from_numpy(np.cos(math.pi - ms)).float().cuda()
        mm = torch.from_numpy(np.sin(math.pi - ms) * ms).float().cuda()
        labels = F.one_hot(labels, out_dim).float()
        logits = logits.float()
        cosine = logits
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * cos_m.view(-1,1) - sine * sin_m.view(-1,1)
        phi = torch.where(cosine > th.view(-1,1), phi, cosine - mm.view(-1,1))
        output = (labels * phi) + ((1.0 - labels) * cosine)
        output *= self.s
        loss = self.crit(output, labels)
        return loss

def set_seed(seed):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)


def get_similiarity_hnsw(embeddings_gallery, emmbeddings_query, k):
    # this is guy is really fast
    print('Processing indices...')

    s = time.time()
    index = faiss.IndexHNSWFlat(embeddings_gallery.shape[1], 32)
    index.add(embeddings_gallery)

    scores, indices = index.search(emmbeddings_query, k)
    e = time.time()

    print(f'Finished processing indices, took {e - s}s')
    return scores, indices

def get_similiarity_l2(embeddings_gallery, emmbeddings_query, k):
    print('Processing indices...')

    s = time.time()
    index = faiss.IndexFlatL2(embeddings_gallery.shape[1])
    index.add(embeddings_gallery)

    scores, indices = index.search(emmbeddings_query, k)
    e = time.time()

    print(f'Finished processing indices, took {e - s}s')
    return scores, indices


def get_similiarity_IP(embeddings_gallery, emmbeddings_query, k):
    print('Processing indices...')

    s = time.time()
    index = faiss.IndexFlatIP(embeddings_gallery.shape[1])
    index.add(embeddings_gallery)

    scores, indices = index.search(emmbeddings_query, k)
    e = time.time()

    print(f'Finished processing indices, took {e - s}s')
    return scores, indices

def get_similiarity(embeddings, k):
    print('Processing indices...')

    index = faiss.IndexFlatL2(embeddings.shape[1])

    res = faiss.StandardGpuResources()

    index = faiss.index_cpu_to_gpu(res, 0, index)

    index.add(embeddings)

    scores, indices = index.search(embeddings, k)
    print('Finished processing indices')

    return scores, indices

def map_per_image(label, predictions, k=5):
    try:
        return 1 / (predictions[:k].index(label) + 1)
    except ValueError:
        return 0.0

def map_per_set(labels, predictions, k=5):
    return np.mean([map_per_image(l, p, k) for l,p in zip(labels, predictions)])

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, window_size=None):
        self.length = 0
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.window_size = window_size

    def reset(self):
        self.length = 0
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        if self.window_size and (self.count >= self.window_size):
            self.reset()
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def get_lr_groups(param_groups):
        groups = sorted(set([param_g['lr'] for param_g in param_groups]))
        groups = ["{:2e}".format(group) for group in groups]
        return groups

def convert_indices_to_labels(indices, labels):
    indices_copy = copy.deepcopy(indices)
    for row in indices_copy:
        for j in range(len(row)):
            row[j] = labels[row[j]]
    return indices_copy

class Multisample_Dropout(nn.Module):
    def __init__(self, dropout_rate=0.1):
        super(Multisample_Dropout, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.dropouts = nn.ModuleList([nn.Dropout((i+1)*.1) for i in range(5)])

    def forward(self, x, module):
        x = self.dropout(x)
        return torch.mean(torch.stack([module(dropout(x)) for dropout in self.dropouts],dim=0),dim=0)

def transforms_auto_augment(image_path, image_size):
    image = Image.open(image_path).convert('RGB')
    train_transforms = transforms.Compose([transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET), transforms.PILToTensor()])
    return train_transforms(image)

def transforms_cutout(image_path, image_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8)
    train_transforms = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.ImageCompression(quality_lower=99, quality_upper=100),
            A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7),
            A.Resize(image_size, image_size),
            A.Cutout(max_h_size=int(image_size * 0.4), max_w_size=int(image_size * 0.4), num_holes=1, p=0.5),
            ToTensorV2(),
        ])
    return train_transforms(image=image)['image']

def transforms_happy_whale(image_path, image_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8)
    aug8p3 = A.OneOf([
            A.Sharpen(p=0.3),
            A.ToGray(p=0.3),
            A.CLAHE(p=0.3),
        ], p=0.5)

    train_transforms = A.Compose([
            A.ShiftScaleRotate(rotate_limit=15, scale_limit=0.1, border_mode=cv2.BORDER_REFLECT, p=0.5),
            A.Resize(image_size, image_size),
            aug8p3,
            A.HorizontalFlip(p=0.5),
            A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
            ToTensorV2(),
        ])
    return train_transforms(image=image)['image']

def transforms_valid(image_path, image_size):
    image = Image.open(image_path).convert('RGB')
    valid_transforms = transforms.Compose([transforms.PILToTensor()])
    return valid_transforms(image)

In [ ]:
class CFG:
    model_name = 'ResNet-50'  # Change the model name
    model_data = ''  # Not needed for ResNet-50
    samples_per_class = 50
    n_classes = 0  # Update this based on your dataset
    min_samples = 4
    image_size = 224
    hidden_layer = 1024  # This parameter is not used for ResNet-50
    seed = 5
    workers = 12
    train_batch_size = 8
    valid_batch_size = 32
    emb_size = 512  # Not used for ResNet-50
    learning_rate = 3e-4  # Adjusted for ResNet-50
    weight_decay = 1e-5  # Adjusted for ResNet-50
    autocast = True
    n_warmup_steps = 1000
    n_epochs = 10
    device = torch.device('cuda')
    s = 30.
    m = .45
    m_min = .05
    acc_steps = 4
    global_step = 0
    reduce_lr = 0.1
    crit = 'ce'


In [ ]:
# Load extensions and set device
%load_ext autoreload
%autoreload 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## Dataset

In [ ]:
# used for training
training_samples = []
values_counts = []
num_classes = 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')
df_g = df.groupby('class', group_keys=True).apply(lambda x: x)


train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
train_df['path'] = train_df.apply(lambda x: '/content/drive/MyDrive/train' + '/' + x['name'], axis=1)


# remove 9397815.jpg from the list!
test_df = pd.read_csv('/content/drive/MyDrive/test_kaggletest.csv')
test_df = test_df.drop(test_df[test_df.name == '9397815.jpg'].index) # smt wrong with this img
test_df['path'] = test_df.apply(lambda x: '/content/drive/MyDrive/test' + '/' + x['name'], axis=1)

df = pd.concat([
    test_df[['class','path']],
    train_df[['class', 'path']]
])
df_g = df.groupby('class', group_keys=True).apply(lambda x: x)


In [ ]:
for group in tqdm(set(df_g['class'])):
    names = list(df_g.path[df_g['class'] == group])
    if len(names) >= CFG.min_samples:
        paths = [
            name for name in names[:CFG.samples_per_class]
        ]

        values_counts.append(len(paths))
        training_samples.extend([
            (p, num_classes) for p in paths
        ])

        num_classes += 1

100%|██████████| 9691/9691 [00:12<00:00, 773.25it/s] 


In [ ]:
data_train = training_samples
value_counts = np.array(values_counts)
CFG.n_classes = num_classes

In [ ]:
len(data_train), CFG.n_classes

(196944, 9691)

## Resnet50 Model

In [ ]:
class Head(nn.Module):
    def __init__(self, hidden_size, n_classes, k=3):
        super(Head, self).__init__()
        self.fc = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        output = self.fc(x)
        return output, F.normalize(x)

class HeadV2(nn.Module):
    def __init__(self, hidden_size, n_classes, k=3):
        super(HeadV2, self).__init__()
        self.fc = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        output = self.fc(x)
        return output, F.normalize(x)

class HeadV3(nn.Module):
    def __init__(self, hidden_size, n_classes, k=3):
        super(HeadV3, self).__init__()
        self.fc = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        output = self.fc(x)
        return output, F.normalize(x)


In [ ]:
class Model(nn.Module):
    def __init__(self, head_size, version='v1', k=3):
        super(Model, self).__init__()
        if version == 'v1':
            self.head = Head(head_size, k)
        elif version == 'v2':
            self.head = HeadV2(head_size, k)
        elif version == 'v3':
            self.head = HeadV3(head_size, k)
        else:
            self.head = Head(head_size, k)

        # Use ResNet-50 as the backbone
        resnet_backbone = models.resnet50(pretrained=True)
        self.encoder = nn.Sequential(*list(resnet_backbone.children())[:-2])

    def forward(self, x):
        x = self.encoder(x)
        x = x.mean([2, 3])  # Global average pooling
        return self.head(x)

    def get_parameters(self):
        parameter_settings = []
        parameter_settings.extend(
            self.get_parameter_section(
                [(n, p) for n, p in self.encoder.named_parameters()],
                lr=CFG.resnet_lr,  # You need to define this learning rate
                wd=CFG.resnet_wd  # You need to define this weight decay
            )
        )

        parameter_settings.extend(
            self.get_parameter_section(
                [(n, p) for n, p in self.head.named_parameters()],
                lr=CFG.hd_lr,
                wd=CFG.hd_wd
            )
        )

        return parameter_settings



    def get_parameter_section(self, parameters, lr=None, wd=None):
      parameter_settings = []

      lr_is_dict = isinstance(lr, dict)
      wd_is_dict = isinstance(wd, dict)

      for no, (n, p) in enumerate(parameters):
        if lr_is_dict:
            temp_lr = next((v for k, v in sorted(lr.items()) if int(k) > no), lr[str(len(lr) - 1)])
        else:
            temp_lr = lr

        if wd_is_dict:
            temp_wd = next((v for k, v in sorted(wd.items()) if int(k) > no), wd[str(len(wd) - 1)])
        else:
            temp_wd = wd

        weight_decay = 0.0 if 'bias' in n else temp_wd

        parameter_setting = {"params": p, "lr": temp_lr, "weight_decay": weight_decay}

        parameter_settings.append(parameter_setting)

        # print(f'no {no} | params {n} | lr {temp_lr} | weight_decay {weight_decay} | requires_grad {p.requires_grad}')
        return parameter_settings


In [ ]:
def ArcFace_criterion(logits_m, target, margins):
    arc = ArcFaceLossAdaptiveMargin(margins=margins, s=CFG.s, crit=CFG.crit)
    loss_m = arc(logits_m, target, CFG.n_classes)
    return loss_m

# Training and Validation

In [ ]:
def train(model, train_loader, optimizer, scaler, scheduler, epoch):
    model.train()
    loss_metrics = AverageMeter()
    criterion = ArcFace_criterion  # You should define this function as mentioned earlier

    tmp = np.sqrt(1 / np.sqrt(value_counts))  # You need to define value_counts
    margins = (tmp - tmp.min()) / (tmp.max() - tmp.min()) * CFG.m + CFG.m_min

    bar = tqdm(train_loader)
    for step, data in enumerate(bar):
        step += 1
        images = data['images'].to(CFG.device, dtype=torch.float)
        labels = data['labels'].to(CFG.device)
        batch_size = labels.size(0)

        with autocast(enabled=CFG.autocast):
            outputs = model(images)  # ResNet-50 model doesn't output features

        loss = criterion(outputs, labels, margins)
        loss_metrics.update(loss.item(), batch_size)
        loss = loss / CFG.acc_steps
        scaler.scale(loss).backward()

        if step % CFG.acc_steps == 0 or step == len(bar):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
            CFG.global_step += 1

        lrs = get_lr_groups(optimizer.param_groups)

        loss_avg = loss_metrics.avg

        bar.set_postfix(loss=loss_avg, epoch=epoch, lrs=lrs, step=CFG.global_step)

@torch.no_grad()
def val(model, valid_loader):
    model.eval()

    all_embeddings = []
    all_labels = []

    for data in tqdm(valid_loader):
        images = data['images'].to(CFG.device, dtype=torch.float)
        labels = data['labels'].to(CFG.device)

        embeddings = model(images)  # ResNet-50 model doesn't output features

        all_embeddings.append(embeddings.detach().cpu().numpy())
        all_labels.append(labels.detach().cpu().numpy())

    all_embeddings = np.concatenate(all_embeddings, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    return all_embeddings, all_labels

In [ ]:
def training(train_loader,
             gallery_loader,
             query_loader,
             experiment_folder,
             version='v1',
             k=3,
             reduce_lr_on_epoch=1,
             use_rampup=True):

    os.makedirs(experiment_folder, exist_ok=True)

    # Initialize ResNet-50 Backbone
    resnet_backbone = models.resnet50(pretrained=True)
    in_features = resnet_backbone.fc.in_features

# Create your Model using the ResNet-50 backbone
    model = Model(in_features, version, k).to(CFG.device)

    optimizer = torch.optim.AdamW(model.get_parameters())

    scaler = torch.cuda.amp.GradScaler(enabled=CFG.autocast)

    steps_per_epoch = math.ceil(len(train_loader) / CFG.acc_steps)

    num_training_steps = math.ceil(CFG.n_epochs * steps_per_epoch)

    if use_rampup:
        scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                    num_training_steps=num_training_steps,
                                                    num_warmup_steps=CFG.n_warmup_steps)
    else:
        scheduler = get_constant_schedule(optimizer)

    best_score = 0
    best_updated_ = 0
    CFG.global_step = 0
    for epoch in range(math.ceil(CFG.n_epochs)):
        print(f'starting epoch {epoch}')


        # train of product-10k
        train(model, train_loader, optimizer, scaler, scheduler, epoch)

        # aicrowd test data
        print('gallery embeddings')
        embeddings_gallery, labels_gallery = val(model, gallery_loader)
        print('query embeddings')
        embeddings_query, labels_query = val(model, query_loader)

        # idk why it is needed
        gc.collect()
        torch.cuda.empty_cache()

        # calculate validation score
        _, indices = utilities.get_similiarity_l2(embeddings_gallery, embeddings_query, 1000)


        indices = indices.tolist()
        labels_gallery = labels_gallery.tolist()
        labels_query = labels_query.tolist()

        preds = convert_indices_to_labels(indices, labels_gallery)
        score = map_per_set(labels_query, preds)
        print('validation score', score)

        # save model
        torch.save({
                'model_state_dict': model.state_dict(),
                }, f'{experiment_folder}/model_epoch_{epoch+1}_mAP3_{score:.2f}.pt')

        # early stopping
        if score > best_score:
            best_updated_ = 0
            best_score = score

        best_updated_ += 1

        if best_updated_ >= 3:
            print('no improvement done training....')
            break

        if (epoch + 1) % reduce_lr_on_epoch == 0:
            scheduler.base_lrs = [g['lr'] * CFG.reduce_lr for g in optimizer.param_groups]

        # to speed up the training
        if epoch > 3:
            break

## Data Loaders for Training and Validation

In [ ]:

def read_img(img_path, is_gray=False):
    mode = cv2.IMREAD_COLOR if not is_gray else cv2.IMREAD_GRAYSCALE
    img = cv2.imread(img_path, mode)
    if not is_gray:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def get_final_transform():
    final_transform = T.Compose([
            T.Resize(
                size=(CFG.image_size, CFG.image_size),
                interpolation=T.InterpolationMode.BICUBIC,
                antialias=True),
            T.ToTensor(),
            T.Normalize(
                mean=(0.48145466, 0.4578275, 0.40821073),
                std=(0.26862954, 0.26130258, 0.27577711)
            )
        ])
    return final_transform

In [ ]:
class ProductDataset(Dataset):
    def __init__(self,
                 data,
                 transform=None,
                 final_transform=None):
        self.data = data
        self.transform = transform
        self.final_transform = final_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        img = read_img(self.data[idx][0])

        if self.transform is not None:
            if isinstance(self.transform, A.Compose):
                img = self.transform(image=img)['image']
            else:
                img = self.transform(img)

        if self.final_transform is not None:
            if isinstance(img, np.ndarray):
                img =  Image.fromarray(img)
            img = self.final_transform(img)

        product_id = self.data[idx][1]
        return {"images": img, "labels": product_id}


In [ ]:
def get_product_10k_dataloader(data_train, data_aug='image_net'):

    transform = None
    if data_aug == 'image_net':
        transform = T.Compose([
            T.ToPILImage(),
            T.AutoAugment(T.AutoAugmentPolicy.IMAGENET)
        ])

    elif data_aug == 'aug_mix':
        transform = T.Compose([
            T.ToPILImage(),
            T.AugMix()
        ])
    elif data_aug == 'happy_whale':
        aug8p3 = A.OneOf([
            A.Sharpen(p=0.3),
            A.ToGray(p=0.3),
            A.CLAHE(p=0.3),
        ], p=0.5)

        transform = A.Compose([
            A.ShiftScaleRotate(rotate_limit=15, scale_limit=0.1, border_mode=cv2.BORDER_REFLECT, p=0.5),
            A.Resize(CFG.image_size, CFG.image_size),
            aug8p3,
            A.HorizontalFlip(p=0.5),
            A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1)
        ])

    elif data_aug == 'cut_out':
        transform = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.ImageCompression(quality_lower=99, quality_upper=100),
            A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7),
            A.Resize(CFG.image_size, CFG.image_size),
            A.Cutout(max_h_size=int(CFG.image_size * 0.4),
                     max_w_size=int(CFG.image_size * 0.4),
                     num_holes=1, p=0.5),
        ])
    elif data_aug == 'clip':
        transform = T.Compose([
            T.ToPILImage(),
            T.RandomResizedCrop(
                size=(224, 224),
                scale=(0.9, 1.0),
                ratio=(0.75, 1.3333),
                interpolation=T.InterpolationMode.BICUBIC,
                antialias=True
            )
        ])
    elif data_aug == 'clip+image_net':
        transform = T.Compose([
            T.ToPILImage(),
            T.AutoAugment(T.AutoAugmentPolicy.IMAGENET),
            T.RandomResizedCrop(
                size=(224, 224),
                scale=(0.9, 1.0),
                ratio=(0.75, 1.3333),
                interpolation=T.InterpolationMode.BICUBIC,
                antialias=True
            )
        ])

    final_transform = get_final_transform()
    train_dataset = ProductDataset(data_train,
                                   transform,
                                   final_transform)
    train_loader = DataLoader(train_dataset,
                              batch_size = CFG.train_batch_size,
                              num_workers=CFG.workers,
                              shuffle=True,
                              drop_last=True)
    print(f'Training Data -> Dataset Length ({len(train_dataset)})')
    return train_loader

In [ ]:
def aicrowd_data_loader(csv_path, img_dir='/content/drive/MyDrive/testing-dataset/development_test_data/development_test_data'):
    df_g = pd.read_csv(csv_path)
    df_g_ = df_g[['img_path', 'product_id']]
    df_g_['img_path'] = df_g_.apply(lambda x: img_dir + '/' + x['img_path'], axis=1)
    data_ = np.array(df_g_).tolist()

    final_transform = get_final_transform()
    dataset = ProductDataset(data_, None, final_transform)
    data_loader = DataLoader(dataset,
                             batch_size = CFG.valid_batch_size,
                             num_workers=CFG.workers,
                             shuffle=False,
                             drop_last=False)

    print(f'{csv_path} -> Dataset Length ({len(dataset)})')
    return data_loader

In [ ]:
gallery_loader = aicrowd_data_loader('/content/drive/MyDrive/testing-dataset/development_test_data/development_test_data/gallery.csv')
query_loader = aicrowd_data_loader('/content/drive/MyDrive/testing-dataset/development_test_data/development_test_data/queries.csv')

<ipython-input-34-935150d57e21>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_g_['img_path'] = df_g_.apply(lambda x: img_dir + '/' + x['img_path'], axis=1)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


/content/drive/MyDrive/testing-dataset/development_test_data/development_test_data/gallery.csv -> Dataset Length (1067)
/content/drive/MyDrive/testing-dataset/development_test_data/development_test_data/queries.csv -> Dataset Length (1935)


<ipython-input-34-935150d57e21>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_g_['img_path'] = df_g_.apply(lambda x: img_dir + '/' + x['img_path'], axis=1)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Training

In [ ]:
k = 3
version = 'v2'
data_aug = 'image_net'
CFG.reduce_lr = 0.1
train_loader = get_product_10k_dataloader(data_train, data_aug)
#experiment_folder = f'my_experiments/{CFG.model_name}-{CFG.model_data}-{str(data_aug)}-{str(version)}-p10k-h&m-Arcface(k={str(k)})-All-Epoch({str(CFG.n_epochs)})-Reduce_LR_0.1'
#training(train_loader,
        # gallery_loader,
         #query_loader,
         #experiment_folder,
         #version=version,
         #k=k)
# idk why it is needed
gc.collect()
torch.cuda.empty_cache()

Training Data -> Dataset Length (196944)
